In [1]:
from __future__ import print_function
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark import SparkContext, SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import array, struct, split, explode, udf, col, collect_list
from  pyspark.sql import functions as f
conf = SparkConf().setMaster("local[*]").setAppName('haha')
sc = SparkContext(conf=conf)
ss = SparkSession(sc)

In [4]:
review_sp = ss.read.json('/Users/Reagan/Downloads/xaa.json').select('asin','reviewText')
reviews = review_sp.groupBy("asin").agg(f.concat_ws(" ", f.collect_list('reviewText')).alias('reviewText'))

In [5]:
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
reviews_w_words = tokenizer.transform(reviews)
remover = StopWordsRemover(inputCol="words", outputCol="mf_words")  # need to add in "book"
reviews_w_mfwords = remover.transform(reviews_w_words)
reviews_w_mfwords.show(3)
# need improvement

+----------+--------------------+--------------------+--------------------+
|      asin|          reviewText|               words|            mf_words|
+----------+--------------------+--------------------+--------------------+
|0001837192|At least it was f...|[at, least, it, w...|[least, first, re...|
|0001845357|Last year I track...|[last, year, i, t...|[last, year, trac...|
|0002216973|And to me, there'...|[and, to, me,, th...|[me,, big, differ...|
+----------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [14]:
from pyspark.sql.functions import monotonically_increasing_id 
df_index = reviews_w_mfwords.select("*").withColumn("id", monotonically_increasing_id())
df_index.show(3)

+----------+--------------------+--------------------+--------------------+---+
|      asin|          reviewText|               words|            mf_words| id|
+----------+--------------------+--------------------+--------------------+---+
|0001837192|At least it was f...|[at, least, it, w...|[least, first, re...|  0|
|0001845357|Last year I track...|[last, year, i, t...|[last, year, trac...|  1|
|0002216973|And to me, there'...|[and, to, me,, th...|[me,, big, differ...|  2|
+----------+--------------------+--------------------+--------------------+---+
only showing top 3 rows



In [7]:
hashingTF = HashingTF(inputCol="mf_words", outputCol="tf", numFeatures=1000)
tf = hashingTF.transform(reviews_w_mfwords)  # reviews_w_feature  == tf
idf = IDF(inputCol="tf", outputCol="tfidf").fit(tf)
tfidf = idf.transform(tf)
tfidf.show(3)

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      asin|          reviewText|               words|            mf_words|                  tf|               tfidf|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0001837192|At least it was f...|[at, least, it, w...|[least, first, re...|(1000,[8,40,65,77...|(1000,[8,40,65,77...|
|0001845357|Last year I track...|[last, year, i, t...|[last, year, trac...|(1000,[1,2,5,9,12...|(1000,[1,2,5,9,12...|
|0002216973|And to me, there'...|[and, to, me,, th...|[me,, big, differ...|(1000,[0,4,5,6,7,...|(1000,[0,4,5,6,7,...|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



# feature vectors

In [45]:
# tfidf.select('tfidf').rdd.map(lambda x: x[0]).collect()

# get the feature matrix

In [ ]:
feature_vecs=tfidf.select('tfidf').rdd.map(lambda x: x[0]).collect()  
full_matrix=[]
for i in range(len(feature_vecs)):
    fea_vec=[]
    for i in range(len(feature_vecs[0])):
        temp = feature_vecs[0][i]
        fea_vec.append(temp)
    full_matrix.append(fea_vec)
